In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
with open("data/ownedgames.json", "r") as f :
    data = json.loads(f.read())

#### Removing empty steamids

In [3]:
data = {key : value for key, value in data.items() if value != {}}

#### Removing zero game_count steamids

In [21]:
data = {key : value for key, value in data1.items() if value["game_count"] != 0}

#### Making a DataFrame

In [22]:
dat = []
for key, value in data.items() :
    steamid = key
    for item in value["games"] :
        dat.append((steamid, item["appid"], item["playtime_forever"]))

In [23]:
df = pd.DataFrame(dat, columns = ["steamid", "appid", "playtime_forever"])

In [24]:
df.head()

,steamid,appid,playtime_forever
0,76561197969918174,10,3
1,76561197969918174,20,0
2,76561197969918174,30,0
3,76561197969918174,40,0
4,76561197969918174,50,0


In [34]:
density = len(df) / (df["steamid"].nunique() * df["appid"].nunique())

In [35]:
print("Unique SteamIds :", df["steamid"].nunique())
print("Unique AppIds :", df["appid"].nunique())
print("Density : {0:.3f} %".format(density * 100))

Unique SteamIds : 16732
Unique AppIds : 26203
Density : 1.458 %
